In [1]:
!rm -fr ~/tmp

In [2]:
from py11 import py11, create_type, set_flags

In [3]:
set_flags("-std=c++17 -L/usr/local/lib64 -lhpx")

In [4]:
import os

In [5]:
os.environ["LD_LIBRARY_PATH"] = "/usr/local/lib64"

In [6]:
create_type("func", "std::function<void()>")

In [7]:
@py11(headers=["<run_hpx.cpp>"])
def hpx_wrapper(f : func )-> None:
    """
    const char * num = getenv("HPX_NUM_THREADS");
    int num_threads = num == 0 ? 4 : atoi(num);
    std::cout << "Using " << num_threads << " threads" << std::endl;
    hpx_global::submit_work(num_threads, f);
    """

g++ -std=c++17 -L/usr/local/lib64 -lhpx -I/usr/include/python3.6m -I/usr/local/include -rdynamic -fPIC -shared -o /home/thomaskahle/tmp/hpx_wrapper_v1.so /home/thomaskahle/tmp/hpx_wrapper.cpp


In [8]:
@py11()
def foo()->None:
    """
        return;
    """

g++ -std=c++17 -L/usr/local/lib64 -lhpx -I/usr/include/python3.6m -I/usr/local/include -rdynamic -fPIC -shared -o /home/thomaskahle/tmp/foo_v1.so /home/thomaskahle/tmp/foo.cpp


In [9]:
@py11(wrap=hpx_wrapper, headers=["<hpx/hpx.hpp>"], funs= [foo])
def hpx_thread_test_function_ptr()->None:
    """
        hpx::thread t1{foo};
        t1.join();
        return;
    """

g++ -std=c++17 -L/usr/local/lib64 -lhpx -I/usr/include/python3.6m -I/usr/local/include -rdynamic -fPIC -shared -o /home/thomaskahle/tmp/hpx_thread_test_function_ptr_v1.so /home/thomaskahle/tmp/hpx_thread_test_function_ptr.cpp


In [10]:
@py11(wrap=hpx_wrapper, headers=["<hpx/hpx.hpp>"])
def hpx_thread_test_lambda()->None:
    """
        hpx::thread t1 {[](){return;}};
        t1.join();
    """

g++ -std=c++17 -L/usr/local/lib64 -lhpx -I/usr/include/python3.6m -I/usr/local/include -rdynamic -fPIC -shared -o /home/thomaskahle/tmp/hpx_thread_test_lambda_v1.so /home/thomaskahle/tmp/hpx_thread_test_lambda.cpp


In [ ]:
@py11(wrap=hpx_wrapper, headers=["hpx/hpx.hpp"], funs = [foo])
def hpx_async_test_function_ptr()->None:
    """
        hpx::lcos::future<void> f1 = hpx::async(foo);
        f1.wait();
    """

In [ ]:
hpx_thread_test_function_ptr() #causes seg fault

In [ ]:
hpx_thread_test_lambda() #also causes seg fault